In [1]:
from hmmlearn import hmm
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')

#### Model

* We have the data about observations states are unknown.  
* * Observation 1 : walk -> shop -> clean
* * Observation 2 : clean -> clean -> clean
* There is a prior probability of a first state in sequence.
* Transition probability : Probability of a next state given current state (never symmetric in general)  
* Emission probability : Probability of observation given state  

These three are the model parameters, which we need to infer.
Our example is discrete HMM and emission probability has multinomial distribution.  
Note that it all sums up to 1.  

In case of continuous HMM emission probabilities can be a normal distribution.  
One example of observed entitity is electric power consumed.  https://nipunbatra.github.io/blog/2013/hmm_continuous.html

In [2]:
states = ('Rainny', 'Sunny')
observations = ('walk', 'shop', 'clean')

start_probability = {
    'Rainy':0.6,
    'Sunny':0.4
}


trainsition_probabilities = {
    'Rainy' : {'Rainy':0.7, 'Sunny':0.3},
    'Sunny' : {'Rainy':0.4, 'Sunny':0.6}
}

emissio_probabilites = {
    'Rainy':{
        'walk':0.1,
        'shop':0.4,
        'clean':0.5
    },
    'Sunny':{
        'walk':0.6,
        'shop':0.3,
        'clean':0.1
    }
}

In [3]:
import numpy as np
from hmmlearn import hmm

model = hmm.MultinomialHMM(n_components=2)
model.startprob_ = np.array([0.6, 0.4])
model.transmat_ = np.array([[0.7,0.3],
                                [0.4,0.6]])
model.emissionprob_ = np.array([[0.1, 0.4, 0.5],
                                [0.6, 0.3, 0.1]])

#### Forward Algorithm

Given model we want to calculate probability of observing sequence.  Brute force approach will start from each state, calculate probability and sums them up. If there are N hidden states and T observations there can be total N^T possible sequence. Forward algorithm uses dynamic programming to reduce complexity. 

In [4]:
prob = math.exp(model.score([2,2,2]))
print "Probability of observing clean -> clean -> clean is", prob

Probability of observing clean -> clean -> clean is 0.045904


In [5]:
prob = math.exp(model.score([0,1,2]))
print "Probability of observing walk -> shop -> clean is", prob

Probability of observing walk -> shop -> clean is 0.033612


In [6]:
prob = math.exp(model.score([1,1]))
print "Probability of observing shop -> shop is", prob

Probability of observing shop -> shop is 0.1296


#### Decoding 

Given observed sequence, we want to find out most probable state sequence.  One way would be as in forward algorithm compute probability of each possible sequence and return the one with highest probability.  
Efficient algorithm here is Viterbi's algorithm which again is based on dynamic programming.  

In [7]:
# Predict the optimal sequence of internal hidden state
X = [1,2,0]
model.decode(X)

(-4.19173690823075, array([0, 0, 1]))

We had observed shop -> clean -> walk.  
States suggested are Rainy, Rainy, Sunny

In [8]:
X = [2,2,2]
print(model.decode(X))

(-3.3036170533232916, array([0, 0, 0]))


We had observed clean -> clean -> clean.  
States suggested are Rainy -> Rainy -> Rainy.  

In [9]:
X = [2,2,2,2,0,0,1,1,2]
print(model.decode(X))

(-11.24525811750575, array([0, 0, 0, 0, 1, 1, 0, 0, 0]))


We had observed clean -> clean -> clean -> clean -> walk -> walk -> shop -> shop -> clean.  
States suggested are Rainy -> Rainy -> Rainy -> Rainy -> Sunny -> Suny -> Rainy -> Rainy -> Rainy

#### Training 

Given dataset of observed sequence we want to learn model parameters.  Algorithm used here is called forward-backward algorithm.  

###### Generating sample data

In [10]:
data = []
for i in range(200):
    X, Z = model.sample(6)
    data.append(X)

We now have 20 possible sequences of length 6.  

In [11]:
for i in range(4):
    print data[i]

[0 1 0 2 1 2]
[1 1 2 2 0 2]
[0 0 2 2 1 1]
[0 0 1 1 1 2]


###### Fitting Model

In [12]:
mm = hmm.MultinomialHMM(n_components=2)
mm.fit(data)

MultinomialHMM(algorithm='viterbi',
        init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
        n_components=2, n_iter=10,
        params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
        random_state=<mtrand.RandomState object at 0x7fb6b03b55f0>,
        startprob=None, startprob_prior=1.0, thresh=0.01, transmat=None,
        transmat_prior=1.0)

In [13]:
mm.emissionprob_

array([[ 0.28162356,  0.34839151,  0.36998493],
       [ 0.37611133,  0.33502647,  0.2888622 ]])

In [14]:
mm.startprob_

array([ 0.49390856,  0.50609144])

In [15]:
mm.transmat_

array([[ 0.4984534 ,  0.5015466 ],
       [ 0.49639699,  0.50360301]])

References :  
* https://web.stanford.edu/~jurafsky/slp3/A.pdf  
* https://medium.com/@kangeugine/hidden-markov-model-7681c22f5b9  

Further reading :  
* We had used library called 'hmmlearn' which seems to be deprecated.  
Library named 'hmms' have great example notebook.  

* https://github.com/lopatovsky/HMMs/blob/master/hmms.ipynb
